In [ ]:
import pywikibot
import igraph as ig
import igraph.remote.gephi as igg

In [ ]:
# Instantiate pywikibot mediawiki as defined in user-config.py
# (potentially leveraging PYWIKIBOT_ROOT/families [look this up])
site = pywikibot.Site()

In [ ]:
page1 = pywikibot.Page('Eps1.1 ones-and-zer0es.mpeg')
page2 = pywikibot.Page('Elliot Alderson')

In [ ]:
page1_attributes = {
    'name': page1.title(), 
    # Note how several attributes/properties are methods, while this is an int
    #  - you can specify to pywikibot.Page whether you want to fetch the content
    'pageid': page1.pageid,
    'revision_count': page1.revision_count(),
    # pywikibot frequently returns objects which may not be serializable
    'namespace': str(page1.namespace()),
    # again, here we convert Category objects into a string
    'categories': ';'.join([category.title() for category in page1.categories()]),
    # Contributors are a dict of usernames and number of revisions
    'contributors': ';'.join(page1.contributors().keys()),
    }

# We will have a function to perform this shortly
page2_attributes = {
    'name': page2.title(), 
    'pageid': page2.pageid,
    'revision_count': page2.revision_count(),
    'namespace': str(page2.namespace()),
    'categories': ';'.join([category.title() for category in page2.categories()]),
    'contributors': ';'.join(page2.contributors().keys()),
    }

In [ ]:
g = ig.Graph()

# Seems igraph doesn't provide much convenience,
# better keep track of vertex IDs (edges will be fine)
vertex_ids = ig.UniqueIdGenerator()

In [ ]:
for attributes in page1_attributes, page2_attributes:
    # Gephi expect vertex/node names in the `Label` field
    attributes['Label'] = attributes['name']
    # igraph.UniqueIdGenerator will retrieve an ID if key exists, or register the next one
    attributes['id'] = vertex_ids[attributes['name']]
    # ... so make sure to add the vertice/vertex if you generate IDs:
    g.add_vertice(1, attributes)

In [ ]:
gephi = igg.GephiConnection()
streamer = igg.GephiGraphStreamer()
streamer.post(g, gephi)

In [ ]:
# import pickle

# with open('mrrobotgraph.pickle', 'wb') as handle:
#     pickle.dump(g, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('mrrobot_idgen.pickle', 'wb') as handle:
#     pickle.dump(id_gen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('mrrobotgraph.pickle', 'rb') as handle:
#     g = pickle.load(handle)

# with open('mrrobot_idgen.pickle', 'rb') as handle:
#     id_gen = pickle.load(handle)

In [ ]:
print(f'vertices: {len(g.vs)}, edges: {len(g.es)}')

print(g.vs[0])
print(g.vs[1])

# print(g)